In [103]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import re
from pymongo import MongoClient
import time
from datetime import datetime

In [104]:
browser = Browser('chrome', headless=False)
time.sleep(1)

In [106]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')
latest_news = soup.find("li", class_="slide")
news_title = latest_news.find("h3").text
news_p = latest_news.find(class_="article_teaser_body").text

In [107]:
news_title

'NASA to Hold Mars 2020 Perseverance Rover Launch Briefing'

In [108]:
news_p

"Learn more about the agency's next Red Planet mission during a live event on June 17."

In [109]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')
base_url = "https://www.jpl.nasa.gov"
style = soup.find(class_="main_feature").find(class_="carousel_items").article["style"]
featured_image_url = base_url + style.split("url")[1].strip(";(')")
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18289-1920x1200.jpg'

In [110]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
time.sleep(1)
xpath = browser.find_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[1]/div/div/div/div/div[2]/section/div/div/div/div[1]/div/div/div/div/article/div/div[2]/div[2]/div[2]/div[1]/div/span')
mars_weather = xpath.text
mars_weather

'InSight sol 552 (2020-06-15) low -91.1ºC (-132.0ºF) high -1.9ºC (28.6ºF)\nwinds from the WNW at 4.4 m/s (9.8 mph) gusting to 15.9 m/s (35.6 mph)\npressure at 7.50 hPa'

In [111]:
url = 'https://space-facts.com/mars/'
table = pd.read_html(url)
space_facts_df = table[2]
space_facts_df.rename(columns={0:'Description', 1:'Value'}, inplace=True)
spaceFactsDF = space_facts_df.set_index('Description')
spaceFactsDF

#table = pd.read_html(url)[0]
#table.rename(columns={0:"metric", 1:"value"}, inplace=True
#table_html = table.to_html(index=False)
#table_html = table_html.replace('\n', '')
#table_html = table_html.replace("<table border=\"1\" class=\"dataframe\">", "").replace("</table>", "").strip()
#table_html

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [112]:
mars_facts_dict = spaceFactsDF.to_dict()
mars_facts_dict

{'Value': {'Equatorial Diameter:': '6,792 km',
  'Polar Diameter:': '6,752 km',
  'Mass:': '6.39 × 10^23 kg (0.11 Earths)',
  'Moons:': '2 (Phobos & Deimos)',
  'Orbit Distance:': '227,943,824 km (1.38 AU)',
  'Orbit Period:': '687 days (1.9 years)',
  'Surface Temperature:': '-87 to -5 °C',
  'First Record:': '2nd millennium BC',
  'Recorded By:': 'Egyptian astronomers'}}

In [113]:
table_html = spaceFactsDF.to_html()
table_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [114]:
table_html = table_html.replace('\n', '')
table_html = table_html.replace("<table border=\"1\" class=\"dataframe\">", "").replace("</table>", "").strip()
table_html

'<thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody>'

In [115]:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg"},
    {"title": "Cerberus Hemisphere", "img_url": "https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg"},
    {"title": "Schiaparelli Hemisphere", "img_url": "https://astrogeology.usgs.gov//cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg"},
    {"title": "Syrtis Major Hemisphere", "img_url": "https://astrogeology.usgs.gov//cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg"},
]

In [116]:
browser.quit()

In [117]:
mongo_client = MongoClient('mongodb://localhost:27017')
db = mongo_client.MarsDB
col = db.scrape_data

In [118]:
datetime_now = datetime.now()

In [119]:
mars_data = {
    "date": datetime_now,
    "news_title": news_title,
    "news_article": news_p,
    "featured_image_urll": featured_image_url,
    "mars_weather": mars_weather,
    "mars_facts": table_html,
    "urls": hemisphere_image_urls
}

print("ndocument body:", mars_data)
result = col.insert_one(mars_data)
print("nresult _id:", result.inserted_id)

ndocument body: {'date': datetime.datetime(2020, 6, 16, 21, 9, 7, 366237), 'news_title': 'NASA to Hold Mars 2020 Perseverance Rover Launch Briefing', 'news_article': "Learn more about the agency's next Red Planet mission during a live event on June 17.", 'featured_image_urll': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18289-1920x1200.jpg', 'mars_weather': 'InSight sol 552 (2020-06-15) low -91.1ºC (-132.0ºF) high -1.9ºC (28.6ºF)\nwinds from the WNW at 4.4 m/s (9.8 mph) gusting to 15.9 m/s (35.6 mph)\npressure at 7.50 hPa', 'mars_facts': '<thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &am